# Database Engineering

All database engineering will be completed within this notebook. SQLAlchemy will be utilized to model table schemas and create a sqlite database for the created tables. 

In [85]:
# Read in dependencies
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime

from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

In [67]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [68]:
# Create a connection to the engine called `conn`
conn = engine.connect()

### Creating the Measurements Table

In [70]:
# Use `declarative_base` from SQLAlchemy to model the measurements table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

class Measurements(Base):
    __tablename__ = 'measurements'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Date)
    prcp = Column(Float)  
    tobs = Column(Float)

In [71]:
# Use `create_all` to create the measurements table in the database
Base.metadata.create_all(engine)

In [72]:
# Load the cleaned csv file into a pandas dataframe
measurements_df = pd.read_csv("Resources/clean_hawaii_measurements.csv")
measurements_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [73]:
# Use Orient='station' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data_measurements = measurements_df.to_dict(orient='records')

In [79]:
# Data is just a list of dictionaries that represent each row of data
data_measurements[0]

{'date': '2010-01-01', 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65}

In [75]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [76]:
# Save the reference to the `stations` table as a variable called `station_table`
measurement_table = sqlalchemy.Table('measurements', metadata, autoload=True)

In [83]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(measurement_table.delete())

In [86]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(measurement_table.insert(), data_measurements)

StatementError: (builtins.TypeError) SQLite Date type only accepts Python date objects as input. [SQL: 'INSERT INTO measurements (station, date, tobs) VALUES (?, ?, ?)'] [parameters: [{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}, {'station': 'USC00519397', 'date': '2010-01-08', 'prcp': 0.0, 'tobs': 64}, {'station': 'USC00519397', 'date': '2010-01-09', 'prcp': 0.0, 'tobs': 68}, {'station': 'USC00519397', 'date': '2010-01-10', 'prcp': 0.0, 'tobs': 73}  ... displaying 10 of 18103 total bound parameter sets ...  {'station': 'USC00516128', 'date': '2017-08-22', 'prcp': 0.5, 'tobs': 76}, {'station': 'USC00516128', 'date': '2017-08-23', 'prcp': 0.45, 'tobs': 76}]]

In [78]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurements limit 5").fetchall()

[]

### Creating the Stations Table

In [19]:
# Use `declarative_base` from SQLAlchemy to model the measurements table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
class Stations(Base):
    __tablename__ = 'stations'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)  
    longitude = Column(Float)
    elevation = Column(Float)

In [20]:
# Use `create_all` to create the stations table in the database
Base.metadata.create_all(engine)

In [21]:
# Load the cleaned csv file into a pandas dataframe
stations_df = pd.read_csv("Resources/clean_hawaii_stations.csv")
stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [89]:
# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
station_data = stations_df.to_dict(orient='records')
station_data[0]

{'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [90]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [91]:
# Save the reference to the `stations` table as a variable called `station_table`
station_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [93]:
# Use `table.delete()` to remove any existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(station_table.delete())

In [94]:
# Use `table.insert()` to insert the data into the table
conn.execute(station_table.insert(), station_data)

In [95]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]